In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import puzz

In [6]:
net = puzz.Network(['a1', 'a2'])
net.add_obstacle('o1')
net.add_obstacle('o2')
net.add_obstacle('o3')
net.add_obstacle('o4')
net.add_ability('a3')
net.add_ability('a4')
net.nodes

set([])
set([])


{'a1': <puzz.Ability at 0x1068693d0>,
 'a2': <puzz.Ability at 0x106869410>,
 'a3': <puzz.Ability at 0x106869550>,
 'a4': <puzz.Ability at 0x106869590>,
 'o1': <puzz.Node at 0x106869310>,
 'o2': <puzz.Node at 0x106869390>,
 'o3': <puzz.Node at 0x106869290>,
 'o4': <puzz.Node at 0x106869510>,
 'start': <puzz.Node at 0x106869350>}

In [7]:
net.defeats('a1', 'o1')
net.defeats('a2', 'o2')
net.defeats('a3', 'o3')
net.defeats('a3', 'o1')
net.defeats('a4', 'o4')

In [12]:
net.nodes['a3'].enabled([])

True

In [11]:
# desired behavior: a3 obsoletes a1. so you should only be able to
# place it after a1, not a2
net.enabled_nodes('a1')
net.enabled_nodes('a2')

{'a3', 'a4', 'o1', 'o3', 'o4'}

{'a3', 'a4', 'o1', 'o3', 'o4'}